<a href="https://colab.research.google.com/github/aderdouri/EiCNAM/blob/master/Tutorials/Notebooks/fd_barrier_option_dupire_sensitivities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mathematical Description of the Financial Model

## Barrier Options under the Black-Scholes Model

A barrier option is a type of financial derivative whose payoff depends on whether the underlying asset's price reaches a specific barrier level during the option's life.


### Black-Scholes Formula for Barrier Options

The value of a barrier option can be derived by adjusting the standard Black-Scholes model with additional terms to account for the barrier conditions. The formulas for *up-and-out call* and *down-and-out put* options are given by:

$
C_{\text{up-out}} =
\begin{cases}
    0, & S \geq H \\\\
    S \Phi(d_1) - K e^{-rT} \Phi(d_2) - \left[ S \left(\frac{H}{S}\right)^{2\lambda} \Phi(x_1) - K e^{-rT} \left(\frac{H}{S}\right)^{2\lambda - 2} \Phi(x_2) \right], & S < H
\end{cases},
$

$
P_{\text{down-out}} =
\begin{cases}
    0, & S \leq H \\\\
    K e^{-rT} \Phi(-d_2) - S \Phi(-d_1) - \left[ K e^{-rT} \left(\frac{H}{S}\right)^{2\lambda - 2} \Phi(-x_2) - S \left(\frac{H}{S}\right)^{2\lambda} \Phi(-x_1) \right], & S > H
\end{cases},
$


### Parameter Definitions

$
d_1 = \frac{\ln(S / K) + (r + 0.5 \sigma^2)T}{\sigma \sqrt{T}}, \quad
d_2 = d_1 - \sigma \sqrt{T},
$

$
x_1 = \frac{\ln(S / H)}{\sigma \sqrt{T}} + \lambda \sigma \sqrt{T}, \quad
x_2 = x_1 - \sigma \sqrt{T},
$

$
\lambda = \frac{r + 0.5 \sigma^2}{\sigma^2}.
$

Here, $ S $ is the spot price, $ K $ is the strike price, $ H $ is the barrier level, $ r $ is the risk-free interest rate, $ T $ is the time to maturity, and $ \sigma $ is the local volatility. $ \Phi(\cdot) $ is the cumulative distribution function of the standard normal distribution.


## Sensitivity Analysis using Automatic Differentiation

The sensitivity of the barrier option price to the local volatility surface $ \sigma_{\text{loc}}(K, T) $ is computed using automatic differentiation. The sensitivity is defined as:

$
\frac{\partial V}{\partial \sigma_{\text{loc}}(K, T)},
$

where $ V $ is the price of the barrier option. By leveraging automatic differentiation, this derivative is computed directly in the computational graph without requiring finite difference approximations.


In [ ]:
import torch
import numpy as np
from torch.distributions.normal import Normal

torch.manual_seed(42)
np.random.seed(42)

torch.set_printoptions(sci_mode=False)

## MONTE CARLO SIMULATION

In [ ]:
def interpolate_volatility(S, t, strikes, times, sigma_surface):
    """
    Interpolate local volatility from the surface using bilinear interpolation.

    Args:
    - S: Spot prices (1D tensor).
    - t: Current time (scalar).
    - strikes: Tensor of strike levels.
    - times: Tensor of time levels.
    - sigma_surface: Local volatility surface (tensor, shape [num_strikes, num_times]).

    Returns:
    - Interpolated volatility values (1D tensor).
    """
    # Ensure inputs are tensors
    S = S.clone().detach()  # Ensure no in-place modifications
    t = t.clone().detach()

    # Find indices for strikes and times
    strike_idx = torch.searchsorted(strikes, S).clamp(1, len(strikes) - 1)
    time_idx = torch.searchsorted(times, t).clamp(1, len(times) - 1)

    # Get bounding indices
    strike_idx0 = strike_idx - 1
    strike_idx1 = strike_idx
    time_idx0 = time_idx - 1
    time_idx1 = time_idx

    # Get bounding values
    S0, S1 = strikes[strike_idx0], strikes[strike_idx1]
    t0, t1 = times[time_idx0], times[time_idx1]

    # Get corresponding volatilities
    vol00 = sigma_surface[strike_idx0, time_idx0]
    vol01 = sigma_surface[strike_idx0, time_idx1]
    vol10 = sigma_surface[strike_idx1, time_idx0]
    vol11 = sigma_surface[strike_idx1, time_idx1]

    # Bilinear interpolation
    vol_t0 = vol00 + (vol10 - vol00) * (S - S0) / (S1 - S0)
    vol_t1 = vol01 + (vol11 - vol01) * (S - S0) / (S1 - S0)
    vol = vol_t0 + (vol_t1 - vol_t0) * (t - t0) / (t1 - t0)

    return vol

In [ ]:
# Monte Carlo simulation for barrier option pricing with sensitivities
def simulate_paths_mc(S0, K, r, sigma_surface, T, H, N_t, N_paths, strikes, times, epsilon=1e-6):
    #dt = torch.tensor(T / N_t, dtype=torch.float32)  # Convert to a PyTorch tensor

    dt = (T / N_t).clone().detach().float()  # Ensure it's a tensor without gradient

    time_grid = torch.linspace(0, T, steps=N_t + 1)
    paths = torch.zeros((N_paths, N_t + 1), dtype=torch.float32, requires_grad=False)
    paths[:, 0] = S0

    alive = torch.ones(N_paths, dtype=torch.float32)  # Track alive paths

    for t_idx in range(1, N_t + 1):
        t = time_grid[t_idx]
        Z = torch.randn(N_paths)  # Standard normal random variables
        S_prev = paths[:, t_idx - 1].clone()  # Clone to avoid in-place modifications

        # Interpolate local volatility for current spot and time
        sigma_t = interpolate_volatility(S_prev, t, strikes, times, sigma_surface)

        # Simulate next step of the path
        dS = r * S_prev * dt + sigma_t * S_prev * torch.sqrt(dt) * Z
        paths[:, t_idx] = S_prev + dS

        # Monitor barrier
        breach_high = paths[:, t_idx] > (H + epsilon)
        breach_low = paths[:, t_idx] < (H - epsilon)
        interpolate_zone = ~(breach_high | breach_low)

        # Update alive status
        alive[breach_high] = 0.0  # Definitely dead
        alive[interpolate_zone] *= 1.0 - (paths[:, t_idx][interpolate_zone] - (H - epsilon)) / (2 * epsilon)

    breached = alive < 1.0  # Paths that breached the barrier
    return paths, breached

## FINITE DIFFERENCES MONTE CARLO

In [ ]:
def finite_difference_sensitivities(S0, K, r, sigma_surface, T, H, N_t, N_paths, strikes, times, epsilon=1e-4, chock=1e-4, option_type="call"):
    """
    Compute sensitivities of the Monte Carlo barrier option price with respect to the local volatility surface
    using finite difference approximation.

    Args:
    - S0: Initial spot price (float or tensor).
    - r: Risk-free rate (float).
    - sigma_surface: Local volatility surface (torch tensor of shape [num_strikes, num_times]).
    - T: Time to maturity (float).
    - H: Barrier level (float).
    - N_t: Number of time steps.
    - N_paths: Number of Monte Carlo paths.
    - strikes: Strike levels (torch tensor).
    - times: Time levels (torch tensor).
    - chock: Small perturbation for finite difference.
    - option_type: "call" or "put".

    Returns:
    - price: Base Monte Carlo price.
    - sensitivities: Sensitivities with respect to the local volatility surface.
    """

    # Base price
    base_paths, base_breached = simulate_paths_mc(S0, K, r, sigma_surface, T, H, N_t, N_paths, strikes, times, epsilon=epsilon)

    terminal_prices = base_paths[:, -1]
    strike_price = K

    if option_type == "call":
        base_payoffs = torch.where(~base_breached, torch.maximum(terminal_prices - strike_price, torch.tensor(0.0)), torch.tensor(0.0))
    elif option_type == "put":
        base_payoffs = torch.where(~base_breached, torch.maximum(strike_price - terminal_prices, torch.tensor(0.0)), torch.tensor(0.0))
    else:
        raise ValueError("Unsupported option type")

    base_price = torch.exp(-r * T) * base_payoffs.mean()

    # Initialize sensitivities
    sensitivities = torch.zeros_like(sigma_surface)

    # Compute finite difference sensitivities
    for i in range(sigma_surface.shape[0]):  # Loop over strikes
        #strike_price = strikes[i]
        for j in range(sigma_surface.shape[1]):  # Loop over times
            # Perturb the volatility surface
            sigma_perturbed = sigma_surface.clone()
            sigma_perturbed[i, j] += chock

            # Perturbed price
            perturbed_paths, perturbed_breached = simulate_paths_mc(S0, K, r, sigma_perturbed, T, H, N_t, N_paths, strikes, times, epsilon=epsilon)
            terminal_prices_perturbed = perturbed_paths[:, -1]

            if option_type == "call":
                perturbed_payoffs = torch.where(~perturbed_breached,
                                                torch.maximum(terminal_prices_perturbed - strike_price, torch.tensor(0.0)), torch.tensor(0.0))
            elif option_type == "put":
                perturbed_payoffs = torch.where(~perturbed_breached,
                                                torch.maximum(strike_price - terminal_prices_perturbed, torch.tensor(0.0)), torch.tensor(0.0))

            perturbed_price = torch.exp(-r * T) * perturbed_payoffs.mean()

            # Sensitivity calculation
            sensitivities[i, j] = (perturbed_price - base_price) / chock

    return base_price.item(), sensitivities

In [ ]:
# Example parameters
S0 = torch.tensor(100.0)  # Spot price
K = torch.tensor(120.0)  # Strike price
H = torch.tensor(150.0)  # Barrier level
T = torch.tensor(2.0)  # Time to maturity (years)
r = torch.tensor(0.0)  # Risk-free rate
N_t = torch.tensor(156)  # Number of time steps (weekly)
N_paths = 100000  # Number of Monte Carlo paths (reduced for testing)
epsilon=0.05*100.0

# Local volatility surface
sigma_surface = torch.ones((10, 5)) * 0.2  # 10 strikes, 5 maturities

# Strike levels and time levels corresponding to the sigma_surface
strikes = torch.linspace(90, 180, 10)
times = torch.linspace(0.1, 2.0, 5)  # Time to maturities

# Calculate sensitivities
price, sensitivities = finite_difference_sensitivities(S0, K, r, sigma_surface, T, H, N_t, N_paths, strikes, times, epsilon=epsilon)

print(f"Monte Carlo Barrier Option Price: {price:.4f}")
print("Finite Difference Sensitivities:")
print(sensitivities)